<a href="https://colab.research.google.com/github/maxhormazabal/depencendy_parsing/blob/main/P2_Preprocessing_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

## Instalación de dependencias

In [ ]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Conectando con Google Drive para leer el archivo `.py` que contiene las funciones a utilizar.

In [ ]:
# Getting access to Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import our own functions (they are in a .py file on Google Drive)
import os
os.chdir("/content/drive/MyDrive/MASTER")
from nlu_preprocessing_utils import *
from conllu import parse
import tensorflow as tf
import pandas as pd
import numpy as np
import math

## Leyendo fuente de datos desde su origen en github

In [ ]:
# English
# 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu'
# https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/en_partut-ud-train.conllu

base_url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/'
file_basename = 'en_partut-ud'
(en_train,en_test,en_val) = readConlluDataset(base_url,file_basename)
en_upo2number, en_number2upo, en_nupos = getUposList(en_train)
number2action,action2number = getActionDict()

152194/152194 [==============================] - 0s 0us/step
Total of different UPOS:  17
{'NOUN': 1, 'ADP': 2, 'DET': 3, 'AUX': 4, 'PART': 5, 'VERB': 6, 'PUNCT': 7, 'PROPN': 8, 'CCONJ': 9, 'ADJ': 10, 'ADV': 11, 'PRON': 12, 'NUM': 13, 'SCONJ': 14, 'X': 15, 'INTJ': 16, 'SYM': 17}


## Transformando la fuente de datos en el dataset inicial

In [ ]:
train_df = conlluToDatasetForDependency(en_train,en_upo2number)
test_df = conlluToDatasetForDependency(en_test,en_upo2number)
val_df = conlluToDatasetForDependency(en_val,en_upo2number)

## Entrenando Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

text = "root"

for sentence in train_df['form']:
  for word in sentence:
    text = text + " " + word

tokenizer = Tokenizer(oov_token="<OOV>",filters="") 
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index

## Post-Oracle datasets

In [ ]:
stack_len = 10
buffer_len = 15

In [ ]:
action_test

array([1, 1, 3, ..., 4, 4, 5])

In [ ]:
deprel_test

array([32., 31.,  0., ...,  0.,  0.,  0.])

In [ ]:
(x_train,action_train,deprel_train) = transformByOracle(train_df,stack_len,buffer_len,en_nupos)
(x_test,action_test,deprel_test) = transformByOracle(test_df,stack_len,buffer_len,en_nupos)
(x_val,action_val,deprel_val) = transformByOracle(val_df,stack_len,buffer_len,en_nupos)

In [ ]:
x_train_clean = np.copy(x_train)
x_test_clean = np.copy(x_test)
x_val_clean = np.copy(x_val)

action_train_clean = np.copy(action_train)
deprel_train_clean = np.copy(deprel_train)
action_test_clean = np.copy(action_test)
deprel_test_clean = np.copy(deprel_test)
action_val_clean = np.copy(action_val)
deprel_val_clean = np.copy(deprel_val)

## X-Variables Tokenization

In [ ]:
x_train_token = applyTokenizer(x_train,stack_len,buffer_len,tokenizer)
x_test_token = applyTokenizer(x_test,stack_len,buffer_len,tokenizer)
x_val_token = applyTokenizer(x_val,stack_len,buffer_len,tokenizer)

## Y-Variables encoding

In [ ]:
number2action

{1: 'Right Arc', 2: 'Left Arc', 3: 'Shift', 4: 'Reduce', 5: 'Done'}

In [ ]:
deprel_train,number2deprel_train,deprel2number_train = deprelToNumerical(deprel_train)
deprel_test,number2deprel_test,deprel2number_test = deprelToNumerical(deprel_test)
deprel_val,number2deprel_val,deprel2number_val = deprelToNumerical(deprel_val)

action_encod_train = tf.keras.utils.to_categorical(action_train)
deprel_encod_train = tf.keras.utils.to_categorical(deprel_train)
action_encod_test = tf.keras.utils.to_categorical(action_test)
deprel_encod_test = tf.keras.utils.to_categorical(deprel_test)
action_encod_val = tf.keras.utils.to_categorical(action_val)
deprel_encod_val = tf.keras.utils.to_categorical(deprel_val)

Creating folder to save the data depending of the size of stack and buffer

In [ ]:
folder_name = str(stack_len)+"stack"+str(buffer_len)+"buffer"
path = "nlu_data/"+folder_name
!mkdir -p {path}

In [ ]:
# function to save the numpy array as a file and re-use it avoiding preprocessing steps
# It will be save in your current directory (seted in "os.chdir("/content/drive/MyDrive/MASTER")")

# Saving train data
np.save(path+'/x_train.npy', x_train_token) 
np.save(path+'/action_train.npy', action_encod_train)
np.save(path+'/deprel_train.npy', deprel_encod_train)

# Saving test data
np.save(path+'/x_test.npy', x_test_token) 
np.save(path+'/action_test.npy', action_encod_test)
np.save(path+'/deprel_test.npy', deprel_encod_test)

# Saving val data
np.save(path+'/x_val.npy', x_val_token) 
np.save(path+'/action_val.npy', action_encod_val)
np.save(path+'/deprel_val.npy', deprel_encod_val)

In [ ]:
# With the following you can read the file and create the numpy array again
new_x_data = np.load('x_data.npy',allow_pickle=True)
new_action_data = np.load('action_data.npy',allow_pickle=True)